In [ ]:
# !pip install opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents langfuse
! pip install aspose.slides

In [ ]:
# requires licence, have water marks on it in free Ver
import aspose.slides as slides
import aspose.slides.charts as charts

with slides.Presentation() as pres:
    # Choose a layout from the master
    layout_slide = pres.masters[0].layout_slides.get_by_type(
        slides.SlideLayoutType.BLANK
    )

    # Add a new slide with that layout
    slide = pres.slides.add_empty_slide(layout_slide)

    # Now you can add a combo chart or other shapes
    chart = slide.shapes.add_chart(
        charts.ChartType.CLUSTERED_COLUMN, 50, 50, 600, 400
    )

    pres.save("combo_editable.pptx", slides.export.SaveFormat.PPTX)


In [ ]:
# no combo chart
from pptx import Presentation
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches

prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[5])

data = CategoryChartData()
data.categories = ['Q1','Q2','Q3','Q4']
data.add_series('Revenue', (120, 150, 170, 210))
data.add_series('Margin', (10, 15, 12, 20))

slide.shapes.add_chart(
    XL_CHART_TYPE.COLUMN_CLUSTERED, Inches(1), Inches(1.5),
    Inches(7), Inches(4),
    data
)

prs.save('editable_chart.pptx')


In [ ]:
# static combo chart 
import io
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches

def create_combo_chart_ppt():
    # --- 1. Create the Combo Chart using Matplotlib ---
    categories = ['Q1', 'Q2', 'Q3', 'Q4']
    bar_data = [120, 150, 170, 210]   # e.g., Revenue
    line_data = [10, 15, 12, 20]      # e.g., Profit Margin

    fig, ax1 = plt.subplots(figsize=(8, 4.5))

    # Bar Plot (Primary Axis)
    ax1.bar(categories, bar_data, color='#4472C4', label='Revenue')
    ax1.set_ylabel('Revenue ($)', color='#4472C4')
    ax1.tick_params(axis='y', labelcolor='#4472C4')

    # Line Plot (Secondary Axis)
    ax2 = ax1.twinx()
    ax2.plot(categories, line_data, color='#ED7D31', linewidth=3, marker='o', label='Margin')
    ax2.set_ylabel('Margin (%)', color='#ED7D31')
    ax2.tick_params(axis='y', labelcolor='#ED7D31')

    plt.title('Quarterly Revenue vs Margin')
    plt.tight_layout()

    # Save chart to a memory buffer
    image_stream = io.BytesIO()
    plt.savefig(image_stream, format='png', dpi=150)
    image_stream.seek(0)
    plt.close()

    # --- 2. Create PowerPoint Presentation ---
    prs = Presentation()
    
    # Use a blank layout (index 6 is usually blank, 5 is Title Only)
    slide_layout = prs.slide_layouts[5] 
    slide = prs.slides.add_slide(slide_layout)

    # Set Title
    if slide.shapes.title:
        slide.shapes.title.text = "Financial Overview"

    # Add the chart image to the slide
    # Position: Left=1 inch, Top=1.5 inches
    slide.shapes.add_picture(image_stream, Inches(1), Inches(1.5), width=Inches(8))

    # Save the file
    output_file = 'combo_chart.pptx'
    prs.save(output_file)
    print(f"Successfully generated {output_file}")

if __name__ == "__main__":
    create_combo_chart_ppt()

Successfully generated combo_chart.pptx


In [14]:
# excel embed approch :
import xlsxwriter

wb = xlsxwriter.Workbook('chart_data.xlsx')
ws = wb.add_worksheet()

# Write data
data = [
    ['Quarter', 'Revenue', 'Margin'],
    ['Q1', 120, 10],
    ['Q2', 150, 15],
    ['Q3', 170, 12],
    ['Q4', 210, 20],
]

for row_idx, row in enumerate(data):
    for col_idx, val in enumerate(row):
        ws.write(row_idx, col_idx, val)

# Create combo chart
chart = wb.add_chart({'type': 'column'})
chart.add_series({
    'name':       '=Sheet1!$B$1',
    'categories': '=Sheet1!$A$2:$A$5',
    'values':     '=Sheet1!$B$2:$B$5',
})
chart.add_series({
    'name':       '=Sheet1!$C$1',
    'categories': '=Sheet1!$A$2:$A$5',
    'values':     '=Sheet1!$C$2:$C$5',
    'type':       'line',
    'y2_axis':    True,
})

chart.set_title({'name': 'Quarterly Revenue vs Margin'})
chart.set_x_axis({'name': 'Quarter'})
chart.set_y_axis({'name': 'Revenue'})
chart.set_y2_axis({'name': 'Margin %'})

ws.insert_chart('E2', chart)
wb.close()


In [16]:
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.shapes import PROG_ID

prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[5])

# Embed the Excel file as an OLE object
slide.shapes.add_ole_object(
    object_file="chart_data.xlsx",  # path to your Excel file
    prog_id=PROG_ID.XLSX,           # correct ProgID for Excel (xlsx) :contentReference[oaicite:1]{index=1}
    left=Inches(1), top=Inches(1),
    width=Inches(2), height=Inches(2)  # this defines the size of the icon
)

prs.save("embedded_excel.pptx")
print("Saved PPT with embedded Excel")


Saved PPT with embedded Excel
